In [ ]:
#we can use roboflox for annotation creation

#we can save data in coco json format and convert this into yolo style format using roboflox
#but if have bit map annotation then that is difficult to do with these tools
#we can convert bitmaps to coco style then to yolo style


In [ ]:
"""Part 1: Convert binary annotations to COCO JSON

For each binary mask, the code extracts contours using OpenCV.
These contours represent the boundaries of objects within the images. This is a key step in converting binary masks to
polygon-like annotations.

Then, convert the contours into annotations, including bounding boxes, area, and segmentation information.
Each annotation is associated with an image ID, category ID, and other properties required by the COCO format.

The code also creates an images section containing metadata about the images, such as their filenames, widths,
and heights. In my example, I have used exactly the same file names for all images and masks so that a given mask
can be easily mapped to the image.

All the annotations, images, and categories are assembled into a dictionary that follows the COCO JSON format.
This includes sections for "info," "licenses," "images," "categories," and "annotations."

Finally, the assembled COCO JSON data is saved to a file, making it ready to be used with tools and frameworks
that support the COCO data format.

Part 2: Converting COCO JSON annotation to YOLO v8

It reads coco style json annotations supplied as a single json file and also
images as input.

Here are the key steps in the code:

1. Convert Images to YOLO Format: The convert_to_yolo function takes paths for input images and annotations
(in JSON format), and directories to store the output images and labels. It then performs the following operations:
- Reads the input JSON file containing annotations.
- Copies all PNG images from the input directory to the output directory.
- Normalizes the polygon segmentation data related to each image and writes them to text files, mapping them to the
 appropriate category (e.g., Alpha, Cells, Mito, Vessels).
- The resulting text files contain information about the object category and the normalized coordinates of the
polygons that describe the objects.

2. Create YAML Configuration File: The create_yaml function takes paths to the input JSON file containing categories,
 training, validation, and optional test paths. It then:
- Extracts the category names and the number of classes.
- Constructs a dictionary containing information about class names, the number of classes, and paths to the training,
validation, and test datasets.
- Writes this dictionary to a YAML file, which can be used as a configuration file for
training a model (e.g., a YOLO model)."""

'Part 1: Convert binary annotations to COCO JSON\n\nFor each binary mask, the code extracts contours using OpenCV. \nThese contours represent the boundaries of objects within the images. This is a key step in converting binary masks to \npolygon-like annotations. \n\nThen, convert the contours into annotations, including bounding boxes, area, and segmentation information. \nEach annotation is associated with an image ID, category ID, and other properties required by the COCO format.\n\nThe code also creates an images section containing metadata about the images, such as their filenames, widths, \nand heights. In my example, I have used exactly the same file names for all images and masks so that a given mask \ncan be easily mapped to the image. \n\nAll the annotations, images, and categories are assembled into a dictionary that follows the COCO JSON format. \nThis includes sections for "info," "licenses," "images," "categories," and "annotations."\n\nFinally, the assembled COCO JSON da

##### for converting bitmap to coco style

In [ ]:
# import glob
# import json
# import os
# import cv2

# # Label IDs of the dataset representing different categories
# category_ids = {
#     "Alpha": 1,
#     "Cells": 2,
#     "Mito": 3,
#     "Vessels": 4,
# }

# MASK_EXT = 'png'
# ORIGINAL_EXT = 'png'
# image_id = 0
# annotation_id = 0

# def images_annotations_info(maskpath):
#     """
#     Process the binary masks and generate images and annotations information.

#     :param maskpath: Path to the directory containing binary masks
#     :return: Tuple containing images info, annotations info, and annotation count
#     """
#     global image_id, annotation_id
#     annotations = []
#     images = []

#     # Iterate through categories and corresponding masks
#     for category in category_ids.keys():
#         for mask_image in glob.glob(os.path.join(maskpath, category, f'*.{MASK_EXT}')):
#             original_file_name = f'{os.path.basename(mask_image).split(".")[0]}.{ORIGINAL_EXT}'
#             mask_image_open = cv2.imread(mask_image)

#             # Get image dimensions
#             height, width, _ = mask_image_open.shape

#             # Create or find existing image annotation
#             if original_file_name not in map(lambda img: img['file_name'], images):
#                 image = {
#                     "id": image_id + 1,
#                     "width": width,
#                     "height": height,
#                     "file_name": original_file_name,
#                 }
#                 images.append(image)
#                 image_id += 1
#             else:
#                 image = [element for element in images if element['file_name'] == original_file_name][0]

#             # Find contours in the mask image
#             gray = cv2.cvtColor(mask_image_open, cv2.COLOR_BGR2GRAY)
#             _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#             contours = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]

#             # Create annotation for each contour
#             for contour in contours:
#                 bbox = cv2.boundingRect(contour)
#                 area = cv2.contourArea(contour)
#                 segmentation = contour.flatten().tolist()

#                 annotation = {
#                     "iscrowd": 0,
#                     "id": annotation_id,
#                     "image_id": image['id'],
#                     "category_id": category_ids[category],
#                     "bbox": bbox,
#                     "area": area,
#                     "segmentation": [segmentation],
#                 }

#                 # Add annotation if area is greater than zero
#                 if area > 0:
#                     annotations.append(annotation)
#                     annotation_id += 1

#     return images, annotations, annotation_id


# def process_masks(mask_path, dest_json):
#     global image_id, annotation_id
#     image_id = 0
#     annotation_id = 0

#     # Initialize the COCO JSON format with categories
#     coco_format = {
#         "info": {},
#         "licenses": [],
#         "images": [],
#         "categories": [{"id": value, "name": key, "supercategory": key} for key, value in category_ids.items()],
#         "annotations": [],
#     }

#     # Create images and annotations sections
#     coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(mask_path)

#     # Save the COCO JSON to a file
#     with open(dest_json, "w") as outfile:
#         json.dump(coco_format, outfile, sort_keys=True, indent=4)

#     print("Created %d annotations for images in folder: %s" % (annotation_cnt, mask_path))

# if __name__ == "__main__":
#     train_mask_path = "EM-platelet-multi/input/train_masks/"
#     train_json_path = "EM-platelet-multi/input/train_images/train.json"
#     process_masks(train_mask_path, train_json_path)

#     val_mask_path = "EM-platelet-multi/input/val_masks/"
#     val_json_path = "EM-platelet-multi/input/val_images/val.json"
#     process_masks(val_mask_path, val_json_path)

##### for converting coco style to yolo style

In [ ]:
# import json
# import os
# import shutil
# import yaml

# # Function to convert images to YOLO format
# def convert_to_yolo(input_images_path, input_json_path, output_images_path, output_labels_path):
#     # Open JSON file containing image annotations
#     f = open(input_json_path)
#     data = json.load(f)
#     f.close()

#     # Create directories for output images and labels
#     os.makedirs(output_images_path, exist_ok=True)
#     os.makedirs(output_labels_path, exist_ok=True)

#     # List to store filenames
#     file_names = []
#     for filename in os.listdir(input_images_path):
#         if filename.endswith(".png"):
#             source = os.path.join(input_images_path, filename)
#             destination = os.path.join(output_images_path, filename)
#             shutil.copy(source, destination)
#             file_names.append(filename)

#     # Function to get image annotations
#     def get_img_ann(image_id):
#         return [ann for ann in data['annotations'] if ann['image_id'] == image_id]

#     # Function to get image data
#     def get_img(filename):
#         return next((img for img in data['images'] if img['file_name'] == filename), None)

#     # Iterate through filenames and process each image
#     for filename in file_names:
#         img = get_img(filename)
#         img_id = img['id']
#         img_w = img['width']
#         img_h = img['height']
#         img_ann = get_img_ann(img_id)

#         # Write normalized polygon data to a text file
#         if img_ann:
#             with open(os.path.join(output_labels_path, f"{os.path.splitext(filename)[0]}.txt"), "a") as file_object:
#                 for ann in img_ann:
#                     current_category = ann['category_id'] - 1
#                     polygon = ann['segmentation'][0]
#                     normalized_polygon = [format(coord / img_w if i % 2 == 0 else coord / img_h, '.6f') for i, coord in enumerate(polygon)]
#                     file_object.write(f"{current_category} " + " ".join(normalized_polygon) + "\n")

# # Function to create a YAML file for the dataset
# def create_yaml(input_json_path, output_yaml_path, train_path, val_path, test_path=None):
#     with open(input_json_path) as f:
#         data = json.load(f)

#     # Extract the category names
#     names = [category['name'] for category in data['categories']]

#     # Number of classes
#     nc = len(names)

#     # Create a dictionary with the required content
#     yaml_data = {
#         'names': names,
#         'nc': nc,
#         'test': test_path if test_path else '',
#         'train': train_path,
#         'val': val_path
#     }

#     # Write the dictionary to a YAML file
#     with open(output_yaml_path, 'w') as file:
#         yaml.dump(yaml_data, file, default_flow_style=False)


# if __name__ == "__main__":
#     base_input_path = "EM-platelet-multi/input/"
#     base_output_path = "EM-platelet-multi/yolo_dataset/"

#     # Processing validation dataset (if needed)
#     convert_to_yolo(
#         input_images_path=os.path.join(base_input_path, "val_images"),
#         input_json_path=os.path.join(base_input_path, "val_images/val.json"),
#         output_images_path=os.path.join(base_output_path, "valid/images"),
#         output_labels_path=os.path.join(base_output_path, "valid/labels")
#     )

#     # Processing training dataset
#     convert_to_yolo(
#         input_images_path=os.path.join(base_input_path, "train_images"),
#         input_json_path=os.path.join(base_input_path, "train_images/train.json"),
#         output_images_path=os.path.join(base_output_path, "train/images"),
#         output_labels_path=os.path.join(base_output_path, "train/labels")
#     )

#     # Creating the YAML configuration file
#     create_yaml(
#         input_json_path=os.path.join(base_input_path, "train_images/train.json"),
#         output_yaml_path=os.path.join(base_output_path, "data.yaml"),
#         train_path="EM-Platelet/train/images",
#         val_path="EM-Platelet/valid/images",
#         test_path='../test/images'  # or None if not applicable
#     )

##### display yolo annotations

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches
# import cv2

# def display_image_with_annotations(image_path, annotation_path, colors=None):
#     # Load image using OpenCV and convert it from BGR to RGB color space
#     image = cv2.imread(image_path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     img_h, img_w, _ = image.shape

#     # Create a figure and axis to display the image
#     fig, ax = plt.subplots(1)
#     ax.imshow(image)
#     ax.axis('off')  # Turn off the axes

#     # Define a default color map if none is provided
#     if colors is None:
#         colors = plt.cm.get_cmap('tab10')

#     # Open the annotation file and process each line
#     with open(annotation_path, 'r') as file:
#         for line in file:
#             parts = line.strip().split()
#             category_id = int(parts[0])
#             # Choose color based on category ID, looping through color map if more than 10 categories
#             color = colors(category_id % 10)
#             # Extract normalized polygon coordinates and denormalize them
#             polygon = [float(coord) for coord in parts[1:]]
#             polygon = [coord * img_w if i % 2 == 0 else coord * img_h for i, coord in enumerate(polygon)]
#             # Reshape into (num_points, 2) array
#             polygon = [(polygon[i], polygon[i+1]) for i in range(0, len(polygon), 2)]
#             # Create a Polygon patch using the denormalized coordinates
#             patch = patches.Polygon(polygon, closed=True, edgecolor=color, fill=False)
#             # Add the patch to the plot to display the annotated region
#             ax.add_patch(patch)

#     plt.show()  # Display the image with annotations

# # Example usage with specified image and annotation paths
# image_path = "EM-platelet-multi/yolo_dataset/train/images/3D-EM-platelet-train04.png"
# annotation_path = "EM-platelet-multi/yolo_dataset/train/labels/3D-EM-platelet-train04.txt"
# display_image_with_annotations(image_path, annotation_path)

In [ ]:
#we need to have yaml file to work with yolo